In [1]:
import pandas as pd
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Preparation of data for Power BI report
---

### Renewable energy source production data sets

- [source](http://data.un.org/Explorer.aspx) for renewable energy data

In [2]:
thermal = pd.read_csv('../data_gdp_and_others/thermal_production_total.csv', parse_dates=['Year'])
wind = pd.read_csv('../data_gdp_and_others/wind_production_total.csv', parse_dates=['Year'])
solar = pd.read_csv('../data_gdp_and_others/solar_electricity_production_total.csv', parse_dates=['Year'])
nuclear = pd.read_csv('../data_gdp_and_others/nuclear_electricity_production_total.csv', parse_dates=['Year'])
ocean = pd.read_csv('../data_gdp_and_others/ocean_electricity_production_total.csv', parse_dates=['Year'])

### Population data set

The next import is a little different and will require some reshaping to fit a join to all the rest
- [source](https://population.un.org/wpp/Download/Standard/CSV/) used for total population (medium variant .csv)

In [3]:
# import and remove extra columns
pop = pd.read_csv('../data_gdp_and_others/world_populations/API_SP.POP.TOTL_DS2_en_csv_v2_511378.csv')
pop.drop(['Country Code', 'Indicator Name', 'Indicator Code', 'Unnamed: 64', '2017', '2018', '2019'], axis=1,inplace=True)
pop = pop.rename(columns={'Country Name': 'country'}).set_index('country')
pop.columns = [int(year) for year in pop.columns]
pop = pop.loc[:,1990:].head()

# transform the dataframe
pop = (
    pop.unstack()
    .swaplevel()
    .to_frame()
    .sort_index()
    .rename(columns={0: 'population'})
)

# name index and reset index for join to main df
pop.index.rename(['country', 'year'], inplace=True)
# pop.reset_index()

---

Function to normalize renewable energy source data sets

In [4]:
def normalize_dfs(df):
    df.columns = ['country', 'Commodity', 'year', 'Unit', df.iloc[0, 1].split(' ')[3].strip(',') + '_mil_kwh', 'Footnotes']
    new = df.iloc[:-2, [0, 2, 4]]
    new.year = pd.to_datetime(new.year).dt.year
    new.set_index(['country', 'year'], inplace=True)
    return new

In [5]:
thermal = normalize_dfs(thermal)
wind = normalize_dfs(wind)
solar = normalize_dfs(solar)
nuclear = normalize_dfs(nuclear)
ocean = normalize_dfs(ocean)

In [6]:
print(f"thermal: {thermal.shape}")
print(f"wind: {wind.shape}")
print(f"solar: {solar.shape}")
print(f"nuclear: {nuclear.shape}")
print(f"ocean: {ocean.shape}")

thermal: (24923, 1)
wind: (1609, 1)
solar: (1268, 1)
nuclear: (821, 1)
ocean: (68, 1)


---

## Join 
- `thermal`
- `wind` 
- `solar` 
- `nuclear` 
- `ocean`
- `pop`

In [7]:
joined = thermal.join([wind, solar, nuclear, ocean], how='outer').fillna(0)
joined.drop_duplicates(inplace=True)
joined['total_kwh'] = joined.sum(axis=1)
joined.head()

thermal_mil_kwh  wind_mil_kwh  solar_mil_kwh  \
country     year                                                 
Afghanistan 1990            364.0           0.0            0.0   
            1991            325.0           0.0            0.0   
            1992            225.0           0.0            0.0   
            1993            220.0           0.0            0.0   
            1994            215.0           0.0            0.0   

                  nuclear_mil_kwh  tide_mil_kwh  total_kwh  
country     year                                            
Afghanistan 1990              0.0           0.0      364.0  
            1991              0.0           0.0      325.0  
            1992              0.0           0.0      225.0  
            1993              0.0           0.0      220.0  
            1994              0.0           0.0      215.0

---
## Test proper export of csv

In [ ]:
joined.to_csv('../data_gdp_and_others/joined_thermal_wind_solar_nuclear_ocean.csv')
df = pd.read_csv('../data_gdp_and_others/joined_thermal_wind_solar_nuclear_ocean.csv')
df.head()

---

## Add columns population and kwh_per_capita and test import

In [8]:
joined = joined.join(pop, how='outer')
joined['kwh_per_capita'] = joined['total_kwh'] / joined['population']
joined.fillna(0)

thermal_mil_kwh  wind_mil_kwh  solar_mil_kwh  \
country     year                                                 
Afghanistan 1990            364.0           0.0            0.0   
            1991            325.0           0.0            0.0   
            1992            225.0           0.0            0.0   
            1993            220.0           0.0            0.0   
            1994            215.0           0.0            0.0   
...                           ...           ...            ...   
Zimbabwe    2014           4397.0           0.0            0.0   
            2015           4582.9           0.0            0.0   
            2015           4542.0           0.0            0.0   
            2016           4174.6           0.0            0.0   
            2016           3908.0           0.0            0.0   

                  nuclear_mil_kwh  tide_mil_kwh  total_kwh  population  \
country     year                                                         
Afghanistan 1990              0.0           0.0      364.0  12412308.0   
            1991              0.0           0.0      325.0  13299017.0   
            1992              0.0           0.0      225.0  14485546.0   
            1993              0.0           0.0      220.0  15816603.0   
            1994              0.0           0.0      215.0  17075727.0   
...                           ...           ...        ...         ...   
Zimbabwe    2014              0.0           0.0     4397.0         0.0   
            2015              0.0           0.0     4582.9         0.0   
            2015              0.0           0.0     4542.0         0.0   
            2016              0.0           0.0     4174.6         0.0   
            2016              0.0           0.0     3908.0         0.0   

                  kwh_per_capita  
country     year                  
Afghanistan 1990        0.000029  
            1991        0.000024  
            1992        0.000016  
            1993        0.000014  
            1994        0.000013  
...                          ...  
Zimbabwe    2014        0.000000  
            2015        0.000000  
            2015        0.000000  
            2016        0.000000  
            2016        0.000000  

[18773 rows x 8 columns]

In [9]:
joined.to_csv('../data_gdp_and_others/joined_thermal_wind_solar_nuclear_ocean_pop_kwh_per_capita.csv')
df = pd.read_csv('../data_gdp_and_others/joined_thermal_wind_solar_nuclear_ocean_pop_kwh_per_capita.csv')
df.shape

(18773, 10)

In [ ]:
df[df.kwh_per_capita.notnull()]